In [21]:
# 구글 코랩에서 모델 학습 진행
# 그리고 코랩에서 비속어 판별 웹 서비스 진행
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.0 MB/s 
     |████████████████████████████████| 6.6 MB 34.6 MB/s 
     |████████████████████████████████| 120 kB 63.5 MB/s 


In [4]:
#! git clone https://github.com/ZIZUN/korean-malicious-comments-dataset.git

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:",device)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

device: cuda:0


데이터셋 제작


In [7]:
df = pd.read_csv("/content/Dataset.csv",sep="\t")
df.head()

,content,lable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   content  10000 non-null  object 
 1   lable    9975 non-null   float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB


In [9]:
df.isna().sum()  # 25개 결측값 확인

content     0
lable      25
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   content  10000 non-null  object 
 1   lable    9975 non-null   float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB


전처리

In [11]:
null_idx = df[df.lable.isnull()].index
df.loc[null_idx, "content"]

1602    응애 응애 엄마 저 맘에 안들죠? ........아들 ?? " 너 내가 우스워 보이...
1654           토니스타크 평소 "아이엠그루트"라는 유행어를 부러워했다는게 학계의 정설\t1
1992    "13일 현대차에 따르면 올 들어 국내 소비자들의 수입차 구매의향률이 3년 만에 하...
2920                 에이프릴이 한마디 합니다 "예쁜게 죄" 구하라님 "무기징역"\t1
3720          답글 글씨체를 봐라 저게 애새끼가 쓴거냐?"빨갱이새끼가 쓴거지 ㅁㅈㅎㅉㅉ\t0
3807    알겠다이기ㅋㅋ 딱 채찍쳐맞는거 좋아하는 한국식 마인드네. 노예마인드. 조금만 성공한...
3908           이래서 스스로 걸리거든 "죄인들이"~ㅎㅎㅎ 재미보고 털리고 그치~~~?\t0
4241    아버지는 내재된 악마들을 다룰 정신적 힘을 가지고 있지 않았다." 이 말한마디가 사...
4283    댓글 중 "선동 당해서 촞불든 개돼지 홍어들도 단죄를 받아야 할 공범자들이다"에10...
5000    스파이 제안받고 살해 안당하는 법1. 처음에 스파이 제안을 받았을때 "중국을 위해서...
5521    "국방부 "까지 ㅡㄱ ㅐ 엿같은 ㅈ ㅣ랄주댕이...좌빨에서 ㅡ인민군대로 ㅡ가려는건가...
5866    쌩뚱맞게 60대최반엌 치매라니 그것도 곱게 사는 사모님이- -" 알콜중독도 아니고 ...
6477    페미메퇘지쿵쾅년인 메갈페미들은 니들이 좋아하는 싫어요 ㄱㄱ제발부탁해~~"일반 여성"...
6538    아니 ㅆㅂ 그런 "카더라"가 넘쳐난다고 그거에 대해서 혹시 댓글게이는 뭔가 아는거 ...
6771    저 때 투니버스에서 코요태 짧게 인터뷰 했었는데 김종민이 "노래는 뭐 신지가 다 하...
6932               개 족 가튼 국방부의 "휴기연장콜센터"발족을 축하한다 ㅆ ㅂ..\t0
7199    민족적 자존심과 애국심을 갖고 국산품 이용합시다 . . . "겸손"한 마음으로 재산...
7252    아나운서는 

In [12]:
# label은 content 가장 끝 문자열로 설정
df.loc[null_idx, "lable"] = df.loc[null_idx, "content"].apply(lambda x: x[-1])

# content "\t" 앞부분까지 문자열로 설정
df.loc[null_idx, "content"] = df.loc[null_idx, "content"].apply(lambda x: x[-2])

In [13]:
df = df.astype({"lable":"int"}) # 학습 위해 int로 변환
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  10000 non-null  object
 1   lable    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


데이터셋 나누기

In [14]:
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

In [15]:
# 데이터셋 갯수 확인
print('중복 제거 전 train: {}.',format(len(train_data)))
print('중복 제거 전 test: {}.',format(len(test_data)))

# 중복 제거
train_data.drop_duplicates(subset=["content"],inplace=True)
test_data.drop_duplicates(subset=["content"],inplace=True)


# 데이터셋 갯수 확인
print('중복 제거 후 train: {}.',format(len(train_data)))
print('중복 제거 후 test: {}.',format(len(test_data)))

중복 제거 전 train: {}. 8000
중복 제거 전 test: {}. 2000
중복 제거 후 train: {}. 7972
중복 제거 후 test: {}. 1997


토크나이징

In [16]:
MODEL_NAME = "beomi/KcELECTRA-base" # hugging face 에 등록된 모델
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [17]:
#학습 데이터 토크나이징
tokenized_train_sentences = tokenizer(
    list(train_data["content"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True,)

In [18]:
print(tokenized_train_sentences[0])
print(tokenized_train_sentences[0].tokens)
print(tokenized_train_sentences[0].ids)
print(tokenized_train_sentences[0].attention_mask)

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '국방부', '~', '~', '전화로', '휴가', '##연장', '##을', '한', '병사', '##들', '몇이나', '되는지', '공개해라', '~', '어느', '훌륭한', '집안', '##의', '자제', '##분들', '##인지도', '같이', '공개해라', '~', '~', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [19]:
# 테스트 데이터 토크나이징
tokenized_test_sentences = tokenizer(
    list(train_data["content"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True,)

In [20]:
print(tokenized_test_sentences[0])
print(tokenized_test_sentences[0].tokens)
print(tokenized_test_sentences[0].ids)
print(tokenized_test_sentences[0].attention_mask)

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '국방부', '~', '~', '전화로', '휴가', '##연장', '##을', '한', '병사', '##들', '몇이나', '되는지', '공개해라', '~', '어느', '훌륭한', '집안', '##의', '자제', '##분들', '##인지도', '같이', '공개해라', '~', '~', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [21]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item

  def __len__(self): 
    return len(self.labels)                       

In [22]:
train_label = train_data["lable"].values
test_label = test_data["lable"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

학습

In [23]:
# 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.de

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

AttributeError: ignored

학습 파라미터 설정

In [24]:
training_args = TrainingArguments(
    output_dir='./',
    num_train_epochs=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    logging_dir='./loss',
    logging_steps=500,
    save_total_limit=2,
)

학습 평가지표 설정


In [25]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average = 'binary')
  acc = accuracy_score(labels,preds)
  return{
      'accuracy':acc,
      'f1':f1,
      'precision':precision,
      'recall':recall
  }

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7972
  Num Epochs = 100
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 99700
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


Step,Training Loss
500,0.039900
1000,0.062700
1500,0.051300
2000,0.043900
2500,0.036000
3000,0.045700
3500,0.034700
4000,0.034300
4500,0.032400
5000,0.037100


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
Deleting older checkpoint [checkpoint-9000] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [checkpoint-9500] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import s

Step,Training Loss
500,0.039900
1000,0.062700
1500,0.051300
2000,0.043900
2500,0.036000
3000,0.045700
3500,0.034700
4000,0.034300
4500,0.032400
5000,0.037100


Saving model checkpoint to ./checkpoint-64000
Configuration saved in ./checkpoint-64000/config.json
Model weights saved in ./checkpoint-64000/pytorch_model.bin
Deleting older checkpoint [checkpoint-63000] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
Saving model checkpoint to ./checkpoint-64500
Configuration saved in ./checkpoint-64500/config.json
Model weights saved in ./checkpoint-64500/pytorch_model.bin
Deleting older checkpoint [checkpoint-63500] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

TrainOutput(global_step=99700, training_loss=0.03498959893807246, metrics={'train_runtime': 14849.5793, 'train_samples_per_second': 53.685, 'train_steps_per_second': 6.714, 'total_flos': 5.2438033333248e+16, 'train_loss': 0.03498959893807246, 'epoch': 100.0})

In [6]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [79]:
torch.save(model, '/content/drive/MyDrive/model_100.pt')

모델평가

In [72]:
trainer.evaluate(eval_dataset=test_dataset)

***** Running Evaluation *****
  Num examples = 1997
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


{'eval_loss': 4.015552043914795,
 'eval_accuracy': 0.5032548823234853,
 'eval_f1': 0.5,
 'eval_precision': 0.4964964964964965,
 'eval_recall': 0.5035532994923858,
 'eval_runtime': 7.2333,
 'eval_samples_per_second': 276.083,
 'eval_steps_per_second': 4.424,
 'epoch': 100.0}

In [73]:

def sentence_predict(sent):
  model.eval() # 평가

  # 입력문장 토크나이징
  tokenized_sent = tokenizer(
      sent,
      return_tensors="pt",
      truncation=True, 
      add_special_tokens=True, 
      max_length=128
      )

  # 모델 위치 gpu이동
  tokenized_sent.to(device)

  # 예측
  with torch.no_grad():
    outputs = model(
        input_ids=tokenized_sent["input_ids"],
        attention_mask=tokenized_sent["attention_mask"],
        token_type_ids=tokenized_sent["token_type_ids"],
    )

  # 결과
  logits = outputs[0]   ## 마지막 노드에서 아무런 Activation Function을 거치지 않은 값을 Logit
  logits = logits.detach().cpu()
  result = logits.argmax(-1)
  if result == 0:
    result = ">> 악성ㅋ"
  elif result ==1:
    result= ">> 정상ㅇ"
  return result
  

while True:
  sentence = input("댓글을 입력 해주세요:")
  if sentence == "0":
      break
  print(sentence_predict(sentence))
  print("\n")

댓글을 입력 해주세요:이 멍청아
>> 악성ㅋ


댓글을 입력 해주세요:새끼야
>> 악성ㅋ


댓글을 입력 해주세요:안뇽
>> 악성ㅋ


댓글을 입력 해주세요:너 천사야?
>> 정상ㅇ


댓글을 입력 해주세요:똥냄새야
>> 악성ㅋ


댓글을 입력 해주세요:너 정말 착하다
>> 정상ㅇ


댓글을 입력 해주세요:ㅏㅇㄴ머하ㅣㅇㅁ;ㅏㅎ
>> 악성ㅋ


댓글을 입력 해주세요:0


ngrok

In [9]:
%%writefile /content/templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">

    <link rel="stylesheet" href="https://pyscript.net/latest/pyscript.css" />
    <script defer src="https://pyscript.net/latest/pyscript.js"></script>
     <py-env>
    - paths:
      - flask_ngrok_example.py
    </py-env>
</head>
    <title>Hello</title>

  
<body>
<h1>욕설감지 챗봇엄나히ㅓㅇㅁㄴ험ㅇ너함어ㅣ;</h1>

 

</html>

Overwriting /content/templates/index.html


In [13]:
# ngrok 설치
!sudo pip3 install flask-ngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from google.colab import drive
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request


In [15]:
 !pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=cc43a38ac48cdb7a64f060d155620010470d9a9813bcaba9b6a61ac4af4aed66
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [16]:
# app.py
from flask import Flask
from flask_ngrok import run_with_ngrok

!ngrok authtoken 2F3a0HVVIP5nQnnOUs31NIbcHpX_4sgq8XrfRfdHwrbSuW1gv

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def hello():
    return render_template('index.html')

if __name__ == '__main__':
    app.run()  # If address is in use, may need to terminate other sessions:
               # Runtime > Manage Sessions > Terminate Other Sessions



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://24b2-35-203-151-205.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [35]:
# 웹에 띄우기 위한 예측평가 함수 - detect.py
from xml.dom.minidom import Element
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import torch


def sentence_predict(sent):
    pt_model = '/content/drive/MyDrive/model_100.pt'
    print(pt_model)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("device:", device)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = torch.load(pt_model, map_location=device)
    print(model)

    MODEL_NAME = "beomi/KcELECTRA-base"  # hugging face 에 등록된 모델
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    model.eval()  # 평가

    # 입력문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 모델 위치 gpu이동
    tokenized_sent.to(device)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"],
        )

    # 결과
    logits = outputs[0]  ## 마지막 노드에서 아무런 Activation Function을 거치지 않은 값을 Logit
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    if result == 0:
        result = sent + ">> 악성ㅋ"

    elif result == 1:
        result = sent + ">> 정상ㅇ"

    return result

# while True:
#   sentence = input("댓글을 입력 해주세요:")
#   if sentence == "0":
#       break
#   print(sentence_predict(sentence))
#   print("\n")

In [38]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok


app = Flask(__name__)
run_with_ngrok(app)


!ngrok authtoken 2F3a0HVVIP5nQnnOUs31NIbcHpX_4sgq8XrfRfdHwrbSuW1gv
@app.route('/', methods=['GET', "POST"])
def main():
    if request.method == 'GET':
        return render_template("index.html")

    if request.method == 'POST':
        # 페이지에서 데이터 받아오기
        sentence = str(request.form['sentence'])

        result = sentence_predict(sentence)

    return render_template("index.html", result=result)


if __name__ == "__main__":
    app.run()



Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://393a-35-203-151-205.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 05:23:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 05:23:41] "GET /favicon.ico HTTP/1.1" 404 -


/content/drive/MyDrive/model_100.pt
device: cuda:0
ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=7

...

INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 06:56:40] "POST / HTTP/1.1" 200 -


/content/drive/MyDrive/model_100.pt
device: cuda:0
ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=7

INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 06:57:16] "POST / HTTP/1.1" 200 -


/content/drive/MyDrive/model_100.pt
device: cuda:0
ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=7

INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 06:58:02] "POST / HTTP/1.1" 200 -
